In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats


flights = pd.read_csv(r"C:\Users\jayxp\Downloads\nycflights.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\jayxp\\Downloads\\nycflights.csv'

## 1. Write a function that takes a single numerical variable and returns three values, the minimum number, the median, and the maximum number of the vector. Test your function using the month column of the flights data set.

In [19]:
def min_median_max(values):
    return np.min(values), np.median(values), np.max(values)

print(min_median_max(flights["month"]))
  

(1, 7.0, 12)


## 2. Explain your reasoning for choosing your function's name in the previous question.

The reason why I chose min_median_max for the function's name is because it does exactly what it does. The function outputs the minimum, median, and maxiumum of the vector.

## 3. Write a function that categorizes a numerical variable in the flights data into four categories..
The function should have two arguments. The first should represent the data object and the second should represent a column name in the data object.

The function should return one new column which categorizes the dep_time column into four categories in the following manner. For any particular variable in the flights data that represents military time (i.e., 0 to 2400 where 1200 represents 12 in the afternoon and 2400 represents midnight), the function should classify values into four categories:

"Morning" for values from 5 am to 11:59 am

"Afternoon" for values from 12 pm to 4:59 pm

"Evening" for values from 5 pm to 8:59 pm

"Night" for values from 9 pm to 4:59 am

Test your function using the dep_time column of the flights dataset. Print a frequency table of the output.

In [23]:
def output_time_of_day(data , column):
    time_of_day = [] #Creating an empty directory 
    for time in data[column]:
        if pd.isna(time):
            time_of_day.append(None) #Append will basically add to the list. 
        elif 500 <= time < 1200:
            time_of_day.append("Morning")
        elif 1200 <= time < 1700:
            time_of_day.append("Afternoon")
        elif 1700 <= time < 2100:
            time_of_day.append("Evening")
        else:
            time_of_day.append("Night")
    data["time_of_day"] = time_of_day  # Add new column to DataFrame
    return data
output_time_of_day(flights, 'dep_time')

#Printing the top 5 and bottom 5 
print(pd.concat([flights[['dep_time', 'time_of_day']].head(5), flights[['dep_time', 'time_of_day']].tail(5)]))

#Also printing the frequency table of the output
print(flights['time_of_day'].value_counts())

      

        dep_time time_of_day
0          517.0     Morning
1          533.0     Morning
2          542.0     Morning
3          544.0     Morning
4          554.0     Morning
336771       NaN        None
336772       NaN        None
336773       NaN        None
336774       NaN        None
336775       NaN        None
time_of_day
Morning      129539
Afternoon     98617
Evening       79793
Night         20572
Name: count, dtype: int64


## 4. Explain your reasoning for choosing your function's name in the previous question.

I chose the function name of 'output_time_of_day' because thats what the function does. The function name is clear and anyone that looks at the function could easily tell that the function will output the time of day. 

## 5. Write a function that calculates the median of all numeric variables in the flights dataset.

In [27]:
## Here I am subsetting the dataframe into numeric columns only, and calling it 'flights_numeric'

In [28]:
def median_numeric_columns (df):
    numeric_variables = df.select_dtypes(include=['number']) ## Subsetting so that the function only includes numeric variables
    column_medians = numeric_variables.median()
    return column_medians
print(median_numeric_columns(flights))

Unnamed: 0        168388.5
year                2013.0
month                  7.0
day                   16.0
dep_time            1401.0
sched_dep_time      1359.0
dep_delay             -2.0
arr_time            1535.0
sched_arr_time      1556.0
arr_delay             -5.0
flight              1496.0
air_time             129.0
distance             872.0
hour                  13.0
minute                29.0
dtype: float64


## 6. Explain your reasoning for choosing your function's name in the previous question.

I chose this name because I wanted a function that could be adapted for other dataframes as well, not just specific to the flights data. The name directly does what it says it does. It finds the median of the numeirc columns only. 

## 7. Modify the function t_test() we wrote in class together this week so that this function can handle violations to the homogeneity of variance (HOV) assumption. This assumption is described below in case you are not familiar with it. Please read the class activity carefully - as all of those details are relevant.

In [44]:
def t_test(num_var, bin_var):  
    
    # Calculate statistics for both groups
    group1 = num_var[bin_var == 0]
    group2 = num_var[bin_var == 1]
   
    n1, n2 = group1.shape[0], group2.shape[0] #Sample sizes
    s2_1, s2_2 = group1.var(), group2.var() #Sample variances
    mean1, mean2 = group1.mean(), group2.mean() #Sample means
    mean_diff = mean2 - mean1 # Calculate the mean difference between the two groups

    if 0.25 <= (s2_1) / (s2_2) <= 4.00:    
        # Standard t-test
        DF = n1 + n2 - 2
        sp = np.sqrt(((n1 - 1) * s2_1 + (n2 - 1) * s2_2) / DF)
        SE_mean_diff = sp * np.sqrt(1/n1 + 1/n2)
        t_statistic = mean_diff / SE_mean_diff
        p_value = 2 * (1 - stats.t.cdf(np.abs(t_statistic), DF))
        test_type = "Independent samples t-test" 
        print("The homogeneity of variance assumption was not violated, so an independent samples t-test was conducted")

    else:
        # Welch's t-test
        SE_mean_diff = np.sqrt(s2_1 / n1 + s2_2 / n2)
        t_statistic = mean_diff / SE_mean_diff
        df = ((s2_1 / n1 + s2_2 / n2) ** 2) / (((s2_1 / n1) ** 2) / (n1 - 1) + ((s2_2 / n2) ** 2) / (n2 - 1))
        p_value = 2 * (1 - stats.t.cdf(np.abs(t_statistic), df))
        test_type = "Welch's t-test"
        print("The homogeneity of variance assumption was violated, so Welch's t-test was conducted.")
        DF = df  # For consistency in the results DataFrame

    results = pd.DataFrame({
        "Continuous Variable": [num_var.name],
        "Binary Variable": [bin_var.name],
        "Total Sample Size": [n1 + n2],
        "Mean Difference": [round(mean_diff, 2)], 
        "SE of Mean Difference": [round(SE_mean_diff, 2)],
        "DF": [DF],
        "t-statistic": [round(t_statistic, 3)],
        "P-value": [("%.3f" % p_value).lstrip('0')],
        "Test": [test_type]
    })
    return results  

In [33]:
## 8.Import the GED data set we used for the class activity. Call the t_test() function and test it out on the GED data set we used in class. Let the numeric variable be 'income_log' and let the binary variable be 'ged'.

In [48]:
ged_data = pd.read_csv(r"C:\Users\jayxp\OneDrive\Desktop\MDS\HOMEWORK THREE MATERIAL\3 ged_data.csv")
t_test(ged_data['income_log'], ged_data['ged'])

The homogeneity of variance assumption was not violated, so an independent samples t-test was conducted


,Continuous Variable,Binary Variable,Total Sample Size,Mean Difference,SE of Mean Difference,DF,t-statistic,P-value,Test
0,income_log,ged,5976,-0.48,0.06,5974,-7.458,.000,Independent samples t-test
